# Gwent Pro Rank analysis

First we'll need to get the data from the playgwent website. Here requests and BeautifulSoup are used to get the data and parse the html the comes back. Nothing to difficult here. There is a little clean up in the code and there are two lines of code to check if a player was in the top 500 last season. In case the data already exists it is loaded from disk.

In [1]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np

# Note that seasons should be in order (for the top 500 spots to be picked up correctly in the next step)
seasons = [
    ('M2_01 Wolf 2020', 'https://masters.playgwent.com/en/rankings/masters-2/season-of-the-wolf/1/{page}', './output/season_of_the_wolf_2020.xlsx'),
    ('M2_02 Love 2020', 'https://masters.playgwent.com/en/rankings/masters-2/season-of-love/1/{page}', './output/season_of_love_2020.xlsx'),
    ('M2_03 Bear 2020', 'https://masters.playgwent.com/en/rankings/masters-2/season-of-the-bear/1/{page}', './output/season_of_the_bear_2020.xlsx'),
    ('M2_04 Elf 2020', 'https://masters.playgwent.com/en/rankings/masters-2/season-of-the-elf/1/{page}', './output/season_of_the_elf_2020.xlsx'),
    ('M2_05 Viper 2020', 'https://masters.playgwent.com/en/rankings/masters-2/season-of-the-viper/1/{page}', './output/season_of_the_viper_2020.xlsx'),
    ('M2_06 Magic 2020', 'https://masters.playgwent.com/en/rankings/masters-2/season-of-the-magic/1/{page}', './output/season_of_magic_2020.xlsx'),
    ('M2_07 Griffin 2020', 'https://masters.playgwent.com/en/rankings/masters-2/season-of-the-griffin/1/{page}', './output/season_of_the_griffin_2020.xlsx'),
    ('M2_08 Draconid 2020', 'https://masters.playgwent.com/en/rankings/masters-2/season-of-the-draconid/1/{page}', './output/season_of_the_draconid_2020.xlsx')
]

os.makedirs('./output', exist_ok=True)

dataframes = []
top500 = []


for season, url_template, output_path in seasons:
    if os.path.exists(output_path):
        df = pd.read_excel(output_path).drop(['Unnamed: 0'], axis=1)
    else:
        output = []

        for i in tqdm(range(1, 150)):
            url = url_template.replace('{page}', str(i))
            try:
                r = requests.get(url)
                soup = BeautifulSoup(r.text, 'html.parser')

                rows = soup.findAll("div", {"class": "c-ranking-table__tr"})
                for row in rows:
                    flag = row.find("i", {"class": "flag-icon"})["class"][1]
                    new_record = {
                        'rank': int(row.find("div", {"class": "td-number"}).text.strip()),
                        'name': row.find("div", {"class": "td-nick"}).text.strip(),
                        'country': flag.replace('flag-icon-', '').upper(),
                        'matches': int(row.find("div", {"class": "td-matches"}).text.strip().replace(' matches', '')),
                        'mmr': int(row.find("div", {"class": "td-mmr"}).text.strip().replace(',', '')),
                        'season': season
                    }
                    output.append(new_record)
            except:
                pass


        df = pd.DataFrame(output).drop_duplicates()
        df.to_excel(output_path)
    
    # We'll add a column here if the player was in the top 500 last season
    df['previous_top500'] = ['yes' if name in top500 else 'no' for name in df['name']]    
    top500 = list(df[df['rank'] <= 500]['name'])
    
    dataframes.append(df)
    
# Concatenate dataframes and drop any records with missing values
full_df = pd.concat(dataframes).dropna()

# Human readable countries

We have the 2 letter code for the country of each participant, let's use the pycountry library to convert that to the full name.

In [2]:
import pycountry

def convert_to_full(two_letter_code):
    try:
        out = pycountry.countries.get(alpha_2=two_letter_code).name
    except:
        print(f'could not convert {two_letter_code}')
        out = two_letter_code
    return out

full_df['country'] = full_df['country'].apply(lambda x: convert_to_full(x))
full_df.head()

,rank,name,country,matches,mmr,season,previous_top500
0,1,kolemoen,Germany,431,10484,M2_01 Wolf 2020,no
1,2,kams134,Poland,923,10477,M2_01 Wolf 2020,no
2,3,TailBot,Poland,538,10472,M2_01 Wolf 2020,no
3,4,Pajabol,Poland,820,10471,M2_01 Wolf 2020,no
4,5,Adzikov,Poland,1105,10442,M2_01 Wolf 2020,no


# National rank

Apart from the global rank, we'll add a participant's national rank as well.

In [3]:
full_df['national_rank'] = full_df.groupby(['country','season'])["mmr"].rank("first", ascending=False)

# Ladder Efficiency

While getting to a higher MMR is the main goal of ladder. There is an argument to be made that the number of games a participant plays to reach a certain point should not be overlooked. Someone that played fewer games to get to a certain MMR climbed the ladder in a more efficient way. There are two metrics that can be used, we take the MMR of a player, substract the minimal MMR, which is 9600, and divide that by the number of matches played. Though, as the higher up you are in ladder, the more fierce the competition becomes, to take this into account Lerio2 from Team Legacy proposed to divide by the square-root of the number of matches instead.

One additional consideration is that players that were in the top 500 the previous season didn't have to climb back up to pro rank. Both metrics actually give them a major advantage. So when comparing efficiency between players the field previous_top500 could be used to allow a fair comparison.

In [4]:
full_df['efficiency'] = ((full_df['mmr']-9600))/full_df['matches']
full_df['lei'] = ((full_df['mmr']-9600))/np.sqrt(full_df['matches'])

In [5]:
full_df.head()

,rank,name,country,matches,mmr,season,previous_top500,national_rank,efficiency,lei
0,1,kolemoen,Germany,431,10484,M2_01 Wolf 2020,no,1.0,2.051044,42.580782
1,2,kams134,Poland,923,10477,M2_01 Wolf 2020,no,1.0,0.950163,28.866807
2,3,TailBot,Poland,538,10472,M2_01 Wolf 2020,no,2.0,1.620818,37.594590
3,4,Pajabol,Poland,820,10471,M2_01 Wolf 2020,no,3.0,1.062195,30.416639
4,5,Adzikov,Poland,1105,10442,M2_01 Wolf 2020,no,4.0,0.761991,25.329753


# Statistics per Season

In [6]:
per_season_df = full_df.groupby(['season']).agg(
    min_mmr     = pd.NamedAgg('mmr', 'min'),
    max_mmr     = pd.NamedAgg('mmr', 'max'),
    num_matches = pd.NamedAgg('matches', 'sum')
).reset_index()

top500_cutoffs = full_df[full_df['rank'] == 500][['season', 'mmr']].rename(columns={'mmr': 'top500_cutoff'})
top200_cutoffs = full_df[full_df['rank'] == 200][['season', 'mmr']].rename(columns={'mmr': 'top200_cutoff'})
top64_cutoffs  = full_df[full_df['rank'] == 64][['season', 'mmr']].rename(columns={'mmr': 'top64_cutoff'})

per_season_df = pd.merge(per_season_df, top500_cutoffs, on='season')
per_season_df = pd.merge(per_season_df, top200_cutoffs, on='season')
per_season_df = pd.merge(per_season_df, top64_cutoffs, on='season')
per_season_df

,season,min_mmr,max_mmr,num_matches,top500_cutoff,top200_cutoff,top64_cutoff
0,M2_01 Wolf 2020,2407,10484,699496,9749,9872,10061
1,M2_02 Love 2020,7776,10537,769358,9832,9952,10117
2,M2_03 Bear 2020,9427,10669,862678,9867,9995,10204
3,M2_04 Elf 2020,9666,10751,1004830,9952,10087,10293
4,M2_05 Viper 2020,9635,10622,859640,9910,10028,10255
5,M2_06 Magic 2020,9624,10597,793401,9896,10002,10191
6,M2_07 Griffin 2020,9698,10667,996742,9978,10100,10289
7,M2_08 Draconid 2020,9666,10546,838212,9946,10061,10246


# Player Summaries

We can summerize player stats over all seasons easily using the code below. We'll store the number of times they were on pro ladder, the worst, best and average MMR scores, ...

In [7]:
player_summaries = full_df.groupby(['name', 'country']).agg(
    appearances = pd.NamedAgg('matches', 'count'),
    min_mmr     = pd.NamedAgg('mmr', 'min'),
    mean_mmr     = pd.NamedAgg('mmr', 'min'),
    max_mmr     = pd.NamedAgg('mmr', 'max'),
    mean_matches = pd.NamedAgg('matches', 'mean'),
    num_matches = pd.NamedAgg('matches', 'sum'),
    best_rank = pd.NamedAgg('rank', 'min'),
    best_national_rank = pd.NamedAgg('national_rank', 'min'),
).reset_index()

player_summaries.head()

,name,country,appearances,min_mmr,mean_mmr,max_mmr,mean_matches,num_matches,best_rank,best_national_rank
0,$Lancer$,China,1,9710,9710,9710,343.000000,343,2011,273.0
1,---Harakternik---,Ukraine,2,9721,9721,9826,176.500000,353,917,57.0
2,--Chimichanga--,Netherlands,3,7406,7406,9816,142.333333,427,629,2.0
3,-17ers-,China,1,9630,9630,9630,172.000000,172,1784,467.0
4,-Adri-,Spain,3,9762,9762,9827,253.666667,761,769,24.0


In [16]:
player_summaries[player_summaries['name'] == 'sepro']

,name,country,appearances,min_mmr,mean_mmr,max_mmr,mean_matches,num_matches,best_rank,best_national_rank
9001,sepro,Belgium,3,9746,9746,9820,242.666667,728,1138,2.0


# Get population for each country

To calculate the number of pro players per capita we'll need the number of inhabitants per country. On Kaggle a dataset was available for 2019. As this doesn't differ much from one year to another (and those will only result into changes several digits behind the decimal point it) the 2019 list is enough for our purpose. The code below loads the data, and generates country names from the two character ISO code using the same package as used before to avoid mismatches.


In [8]:
# data from kaggle https://www.kaggle.com/erikbruin/countries-of-the-world-iso-codes-and-population/data#
# the number are indicated very strangely in the csv with a decimal point and need to be multiplied with 1000

population_df = pd.read_csv('./data/countries_by_population_2019.csv')[['name', 'pop2019']]
country_codes = pd.read_csv('./data/country_codes_2020.csv')[['name', 'cca2']]
population_df = pd.merge(population_df, country_codes, how='left', left_on='name', right_on='name')

population_df['country'] = population_df['cca2'].apply(lambda x: convert_to_full(x))
population_df['pop2019'] = (population_df['pop2019'] * 1000).astype(int)

could not convert nan


In [9]:
# Lets do some counts per season and country
national_stats = full_df.groupby(['season','country']).agg(
    total_matches = pd.NamedAgg('matches', 'sum'),
    num_players = pd.NamedAgg('name', 'count')
).reset_index()

# Add the population (from 2019) to each country and calculate the number of pro players (top 2860) per million inhabitants
national_stats = pd.merge(national_stats, population_df.drop(['name', 'cca2'], axis=1), on='country', how='left')
national_stats['pro_players_per_million'] = (national_stats['num_players']*1000000)/national_stats['pop2019']
national_stats['matches_per_player'] = national_stats['total_matches']/national_stats['num_players']

national_stats

,season,country,total_matches,num_players,pop2019,pro_players_per_million,matches_per_player
0,M2_01 Wolf 2020,Algeria,250,1,43053054.0,0.023227,250.000000
1,M2_01 Wolf 2020,Argentina,1480,4,44780677.0,0.089324,370.000000
2,M2_01 Wolf 2020,Armenia,546,1,2957731.0,0.338097,546.000000
3,M2_01 Wolf 2020,Australia,6193,27,25203198.0,1.071293,229.370370
4,M2_01 Wolf 2020,Austria,3981,16,8955102.0,1.786691,248.812500
...,...,...,...,...,...,...,...
678,M2_08 Draconid 2020,United Kingdom,27540,90,67530172.0,1.332738,306.000000
679,M2_08 Draconid 2020,United States,56490,194,329064917.0,0.589549,291.185567
680,M2_08 Draconid 2020,Uzbekistan,778,3,32981716.0,0.090959,259.333333
681,M2_08 Draconid 2020,"Venezuela, Bolivarian Republic of",814,3,28515829.0,0.105205,271.333333


# Adding popularity metrics


We can now calculate the number of pro players per million inhabitants, this could be a proxy for Gwent's popularity in 
different parts of the world

In [10]:
top_popularity = national_stats[national_stats['season'] == 'M2_08 Draconid 2020'].sort_values('pro_players_per_million', ascending=False).drop(columns=['pop2019'])
top_popularity.head(10)

,season,country,total_matches,num_players,pro_players_per_million,matches_per_player
657,M2_08 Draconid 2020,Poland,72225,267,7.047129,270.505618
617,M2_08 Draconid 2020,Estonia,1726,7,5.280436,246.571429
660,M2_08 Draconid 2020,Russian Federation,195905,673,4.613626,291.092125
599,M2_08 Draconid 2020,Belarus,10260,39,4.125931,263.076923
677,M2_08 Draconid 2020,Ukraine,52333,162,3.682351,323.043210
626,M2_08 Draconid 2020,Iceland,307,1,2.949583,307.000000
618,M2_08 Draconid 2020,Finland,4052,14,2.530659,289.428571
624,M2_08 Draconid 2020,Hong Kong,5154,18,2.420606,286.333333
644,M2_08 Draconid 2020,Malta,229,1,2.270807,229.000000
610,M2_08 Draconid 2020,Croatia,2247,9,2.179016,249.666667


An alternative way is to find places where the die-hard players are located, those that play the most games per season on average.

In [11]:
top_popularity = national_stats[national_stats['season'] == 'M2_08 Draconid 2020'].sort_values('matches_per_player', ascending=False).drop(columns=['pop2019'])
top_popularity.head(10)

,season,country,total_matches,num_players,pro_players_per_million,matches_per_player
611,M2_08 Draconid 2020,Cyprus,1217,2,1.668648,608.500000
608,M2_08 Draconid 2020,"Congo, The Democratic Republic of the",570,1,0.011522,570.000000
593,M2_08 Draconid 2020,Armenia,1002,2,0.676194,501.000000
631,M2_08 Draconid 2020,Ireland,2988,6,1.228880,498.000000
596,M2_08 Draconid 2020,Azerbaijan,1447,3,0.298575,482.333333
654,M2_08 Draconid 2020,Panama,453,1,0.235491,453.000000
669,M2_08 Draconid 2020,Switzerland,2580,6,0.698376,430.000000
603,M2_08 Draconid 2020,Bulgaria,5516,13,1.857111,424.307692
662,M2_08 Draconid 2020,Serbia,4074,10,1.139960,407.400000
656,M2_08 Draconid 2020,Philippines,5347,14,0.129490,381.928571


# Creating a ranking of nations

We'll select countries with three or more playing in the pro rank, select the top 3 players and calculate the average MMR and total MMR for the top
3 players of the country.



In [12]:
num_player_included = 3   # Here we define the number of players in Pro Ladder to be considered

players_per_country = full_df.groupby(['season','country']).agg(
    num_players = pd.NamedAgg('name', 'count')
).reset_index()

players_per_country = players_per_country[players_per_country['num_players'] >= num_player_included]

top_per_country = pd.merge(players_per_country[['season', 'country']], full_df, how='inner', on=['season', 'country'])
top_per_country = top_per_country[top_per_country['national_rank'] <= num_player_included]
top_per_country

,season,country,rank,name,matches,mmr,previous_top500,national_rank,efficiency,lei
0,M2_01 Wolf 2020,Argentina,174,Srchenko,393,9887,no,1.0,0.730280,14.477235
1,M2_01 Wolf 2020,Argentina,471,eltroco,339,9758,no,2.0,0.466077,8.581382
2,M2_01 Wolf 2020,Argentina,846,MorgannFreemann,208,9660,no,3.0,0.288462,4.160251
4,M2_01 Wolf 2020,Australia,135,whitecool,404,9914,no,1.0,0.777228,15.622084
5,M2_01 Wolf 2020,Australia,211,Pacifier3750,453,9863,no,2.0,0.580574,12.356818
...,...,...,...,...,...,...,...,...,...,...
22581,M2_08 Draconid 2020,"Venezuela, Bolivarian Republic of",2032,CJKAPA,276,9709,no,2.0,0.394928,6.561029
22582,M2_08 Draconid 2020,"Venezuela, Bolivarian Republic of",2684,podosqui,189,9674,no,3.0,0.391534,5.382708
22583,M2_08 Draconid 2020,Viet Nam,215,umeu,697,10052,yes,1.0,0.648494,17.120721
22584,M2_08 Draconid 2020,Viet Nam,642,huynguyenquang,205,9890,no,2.0,1.414634,20.254479


In [13]:
top_stats = top_per_country.groupby(['season','country']).agg(
    mean_mmr                = pd.NamedAgg('mmr', 'mean'),
    total_mmr               = pd.NamedAgg('mmr', 'sum'),
    mean_matches_per_player = pd.NamedAgg('matches', 'mean'),
    total_matches           = pd.NamedAgg('matches', 'sum')
).reset_index()

# Here we add the same ranking and effeciency statistics for national teams as for individual players
top_stats['nation_rank'] = top_stats.groupby(['season'])["mean_mmr"].rank("first", ascending=False)
top_stats['efficiency'] = ((top_stats['mean_mmr']-9600))/top_stats['mean_matches_per_player']
top_stats['lei'] = ((top_stats['mean_mmr']-9600))/np.sqrt(top_stats['mean_matches_per_player'])

top_stats

,season,country,mean_mmr,total_mmr,mean_matches_per_player,total_matches,nation_rank,efficiency,lei
0,M2_01 Wolf 2020,Argentina,9768.333333,29305,313.333333,940,34.0,0.537234,9.509700
1,M2_01 Wolf 2020,Australia,9869.666667,29609,336.333333,1009,22.0,0.801784,14.704231
2,M2_01 Wolf 2020,Austria,9905.666667,29717,392.333333,1177,19.0,0.779099,15.431938
3,M2_01 Wolf 2020,Belarus,10064.333333,30193,390.000000,1170,11.0,1.190598,23.512432
4,M2_01 Wolf 2020,Belgium,9901.000000,29703,415.666667,1247,21.0,0.724138,14.763655
...,...,...,...,...,...,...,...,...,...
480,M2_08 Draconid 2020,United Kingdom,10284.333333,30853,625.666667,1877,8.0,1.093767,27.358746
481,M2_08 Draconid 2020,United States,10300.333333,30901,463.666667,1391,7.0,1.510424,32.523843
482,M2_08 Draconid 2020,Uzbekistan,9854.666667,29564,259.333333,778,52.0,0.982005,15.814044
483,M2_08 Draconid 2020,"Venezuela, Bolivarian Republic of",9766.000000,29298,271.333333,814,63.0,0.611794,10.077586


In [14]:
top_stats[top_stats['season'] == 'M2_08 Draconid 2020'].sort_values('nation_rank')

,season,country,mean_mmr,total_mmr,mean_matches_per_player,total_matches,nation_rank,efficiency,lei
432,M2_08 Draconid 2020,China,10489.333333,31468,409.000000,1227,1.0,2.174409,43.974703
466,M2_08 Draconid 2020,Russian Federation,10479.666667,31439,636.000000,1908,2.0,1.383124,34.881052
463,M2_08 Draconid 2020,Poland,10439.333333,31318,657.000000,1971,3.0,1.277524,32.745512
440,M2_08 Draconid 2020,Germany,10399.666667,31199,440.000000,1320,4.0,1.817424,38.122613
450,M2_08 Draconid 2020,Japan,10345.000000,31035,708.666667,2126,5.0,1.051270,27.985642
...,...,...,...,...,...,...,...,...,...
446,M2_08 Draconid 2020,"Iran, Islamic Republic of",9787.000000,29361,264.000000,792,60.0,0.708333,11.509054
431,M2_08 Draconid 2020,Chile,9782.666667,29348,310.000000,930,61.0,0.589247,10.374770
469,M2_08 Draconid 2020,Singapore,9772.666667,29318,325.000000,975,62.0,0.531282,9.577823
483,M2_08 Draconid 2020,"Venezuela, Bolivarian Republic of",9766.000000,29298,271.333333,814,63.0,0.611794,10.077586


# Export everything

Lets exports all data to Excel files so they can be further analysed without pandas/python. 

In [15]:
full_df.to_excel('./output/player_stats.xlsx')
national_stats.to_excel('./output/national_stats.xlsx')
per_season_df.to_excel('./output/seasonal_stats.xlsx')
player_summaries.to_excel('./output/player_summaries.xlsx')